# Part 2: Power Method

In [51]:
import numpy as np

#### 1. Implement the power method with normalization.
Here we've implemented the following algorithm:
- Take a random vector $x^{(0)}$.
- For $k = 1, 2, ..., i$ 
  - Max:
    - $z^{(k+1)} = Ax^{(k)}$
    - $x^{(k+1)} = \frac{z^{(k+1)}}{||z^{(k+1)}||}$
    - Look for $i$ such that $z_i^{(k+1)}$ is the largest element in the vector $z^{(k+1)}$.
    - $\lambda^{(k+1)} = \frac{z_i^{(k+1)}}{x_i^{(k)}}$

This algorithm will be applied on the following matrix, after which the calculated eigenvalues with their corresponding eigenvectors will be printed.

$$
A = \begin{bmatrix}
5 & -1 & -2 & 0\\
-1 & 3 & -2 & -1 \\
-2 & -2 & 5 & 0 \\
0 & -1 & 0 & 4
\end{bmatrix}
$$

In [52]:
def power_method(matrix: np.ndarray, maximum_iterations: int):
    x_k = np.random.rand(matrix.shape[1])
    lmbda = 0
    for _ in range(maximum_iterations):
        z_k1 = np.dot(matrix, x_k)
        x_k = z_k1 / np.linalg.norm(z_k1)
        i = np.argmax(z_k1)
        lmbda = z_k1[i]/x_k[i]

    return lmbda, x_k
A = np.array([[5, -1, -2, 0], [-1, 3, -2, -1], [-2, -2, 5, 0], [0, -1, 0, 4]])
eigenvalue, eigenvector = power_method(A, 100)
print(f"Largest (absolute) eigenvalue: {eigenvalue}, associated eigenvector: {eigenvector}")
print(f"Ax = {A @ eigenvector} = lambda x = {eigenvalue * eigenvector}")

Largest (absolute) eigenvalue: 7.179335040371525, associated eigenvector: [-0.59084255 -0.24322828  0.76543608  0.07650288]
Ax = [-4.24185663 -1.74621733  5.49532206  0.5492398 ] = lambda x = [-4.24185663 -1.74621733  5.49532206  0.5492398 ]


#### 2. Extend your function, to a new one, such that it takes an extra argument $\epsilon = 10^{−5}$. Keep track on the number of iterations that you need to reach the precision $|λ(k+1) − λ(k)| ≤ ε$.
Here we've used the same function as the previous excercize, with the added ability to specify an argument $\epsilon$ to dictate the wanted precision. The for loop which contains our algorithm breaks (stops) when the difference between eigenvalue $k-1$ and eigenvalue $k$ is smaller than $\epsilon$. We also added a new variable ```itr``` to keep track of the iterations.


In [53]:
def power_method_limit(matrix: np.ndarray, maximum_iterations: int, epsln: float):
    x_k = np.random.rand(matrix.shape[1])
    oldlmbda = 0
    lmbda = 0
    itr = 0
    
    for _ in range(maximum_iterations):
        z_k1 = np.dot(matrix, x_k)
        x_k = z_k1 / np.linalg.norm(z_k1)
        i = np.argmax(z_k1)
        oldlmbda = lmbda
        lmbda = z_k1[i]/x_k[i]
        itr += 1
        if (np.abs(lmbda - oldlmbda) <= epsln):
            break

    return lmbda, x_k, itr
A = np.array([[5, -1, -2, 0], [-1, 3, -2, -1], [-2, -2, 5, 0], [0, -1, 0, 4]])
eigenvalue_lim, eigenvector_lim, iterations_lim = power_method_limit(A, 100, 1e-5)
print(f"Largest (absolute) eigenvalue: {eigenvalue_lim}, associated eigenvector: {eigenvector_lim}, iterations: {iterations_lim}")
print(f"Ax = {A @ eigenvector_lim} = lambda x = {eigenvalue_lim * eigenvector_lim}")


Largest (absolute) eigenvalue: 7.179327213465006, associated eigenvector: [ 0.59181702  0.24219264 -0.76508741 -0.07574036], iterations: 18
Ax = [ 4.24706727  1.74067607 -5.49345634 -0.54515409] = lambda x = [ 4.24884803  1.73878018 -5.49281284 -0.54376486]


#### 3. Apply your code to find the eigenvalue of $A$ with the smallest absolute value.
We've used the inverse power method to find the smallest eigenvalue. For this we first found the inverse of $A$: $A^{-1}$. We then applied the standard power method on $A^{-1}$ to find the smallest eigenvalue and it's corresponding eigenvector.


In [54]:
def power_method_inverse(matrix: np.ndarray, maximum_iterations: int, epsln: float):
    inverse_matrix = np.linalg.inv(matrix)
    x_k = np.random.rand(inverse_matrix.shape[1])
    oldlmbda = 0
    lmbda = 0
    itr = 0
    
    for _ in range(maximum_iterations):
        z_k1 = np.dot(inverse_matrix, x_k)
        x_k = z_k1 / np.linalg.norm(z_k1)
        i = np.argmax(z_k1)
        oldlmbda = lmbda
        lmbda = z_k1[i]/x_k[i]
        itr += 1
        if (np.abs(lmbda - oldlmbda) <= epsln):
            break
    smallest_lbmda = 1/lmbda
    return smallest_lbmda, x_k, itr
A = np.array([[5, -1, -2, 0], [-1, 3, -2, -1], [-2, -2, 5, 0], [0, -1, 0, 4]])
eigenvalue_inv, eigenvector_inv, iterations_inv = power_method_inverse(A, 100, 1e-5)
print(f"Smalles (absolute) eigenvalue: {eigenvalue_inv}, associated eigenvector: {eigenvector_inv}, iterations: {iterations_inv}")
print(f"Ax = {A @ eigenvector_inv} = lambda x = {eigenvalue_inv * eigenvector_inv}")

Smalles (absolute) eigenvalue: 0.6800134632155518, associated eigenvector: [0.40746162 0.71808884 0.52108295 0.21632384], iterations: 6
Ax = [0.27705338 0.48831518 0.35431379 0.14720651] = lambda x = [0.27707939 0.48831008 0.35434342 0.14710312]


#### 4. Apply your code to find the eigenvalue of A that is closest to 3.

We've used the shifted inverse power method to find the eigenvalue closest to 3. We first shifted the matrix by $s$, which is a new variable that can be specified. We then applied the inverse power method to find the eigenvalue closest to 3.

In [55]:
def power_method_shifted(matrix: np.ndarray, maximum_iterations: int, epsln: float, shift: float):
    matrix = matrix - shift
    inverse_matrix = np.linalg.inv(matrix)
    x_k = np.random.rand(inverse_matrix.shape[1])
    oldlmbda = 0
    lmbda = 0
    itr = 0
    
    for _ in range(maximum_iterations):
        z_k1 = np.dot(inverse_matrix, x_k)
        x_k = z_k1 / np.linalg.norm(z_k1)
        i = np.argmax(z_k1)
        oldlmbda = lmbda
        lmbda = z_k1[i]/x_k[i]
        itr += 1
        if (np.abs(lmbda - oldlmbda) <= epsln):
            break
    smallest_lbmda = 1/lmbda
    return smallest_lbmda, x_k, itr

A = np.array([[5, -1, -2, 0], [-1, 3, -2, -1], [-2, -2, 5, 0], [0, -1, 0, 4]])
eigenvalue_shift, eigenvector_shift, iterations_shift = power_method_shifted(A, 100, 1e-10, 3)
print(f"Closest eigenvalue to 3: {eigenvalue_shift}")

Closest eigenvalue to 3: 3.690000971986961
